In [1]:
import pandas as pd
import pandana as pdna
from pandana.loaders import osm
import numpy as np
import warnings
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
import osmnx as ox
import os
import timeit

ox.settings.use_cache = True

ox.settings.log_console = True

In [ ]:
start = timeit.timeit() #timing
#osmnx
filepath = "./data/prague_network.graphml"
if(os.path.exists(filepath)):
    G = ox.io.load_graphml(filepath)
else:
    #bbox = 12.4763, 55.5504, 12.6878, 55.7016
    #G = ox.graph.graph_from_bbox(bbox, simplify = True, network_type="walk")  
    G = ox.graph.graph_from_place("Prague, Czechia")
    ox.io.save_graphml(G, filepath)

G = ox.project_graph(G)

#pandana
filepath_pan = "./data/prague_features.h5"

if(os.path.exists(filepath_pan)):
    n, e = ox.graph_to_gdfs(G)
    e = e.reset_index()
    network = pdna.Network.from_hdf5(filepath_pan)
else:
    n, e = ox.graph_to_gdfs(G)
    e = e.reset_index()
    network = pdna.Network(n.geometry.x, n.geometry.y, e["u"], e["v"], e[["length"]])
    
    network.save_hdf5(filepath_pan)

places = ["Prague, Czechia",]

pois_education = ox.features_from_place(
    places,
    tags={"amenity": "school"},
).to_crs(n.crs)

pois_health = ox.features_from_place(
    places,
    tags={"amenity": "pharmacy", "amenity": "clinic"},
).to_crs(n.crs)

pois_basic = ox.features_from_place(
    places,
    tags={"amenity": "bus_station", "shop": "supermarket", "shop": "convenience"},
).to_crs(n.crs)

pois_education = pois_education.copy()
pois_education["geometry"] = pois_education.geometry.centroid
pois_health = pois_health.copy()
pois_health["geometry"] = pois_health.geometry.centroid
pois_basic = pois_basic.copy()
pois_basic["geometry"] = pois_basic.geometry.centroid
dist = 1000

network.set_pois(
    category="pois_education",
    maxdist=2000,
    maxitems=1000,
    x_col=pois_education.geometry.x,
    y_col=pois_education.geometry.y,
)

network.set_pois(
    category="pois_health",
    maxdist=2000,
    maxitems=1000,
    x_col=pois_health.geometry.x,
    y_col=pois_health.geometry.y,
)

network.set_pois(
    category="pois_basic",
    maxdist=2000,
    maxitems=1000,
    x_col=pois_basic.geometry.x,
    y_col=pois_basic.geometry.y,
)


nearest_pois = network.nearest_pois(
    distance=2000,
    category="pois_education",
    num_pois=1,
)
n["pois"] = (nearest_pois <= dist).sum(axis=1)


nearest_pois = network.nearest_pois(
    distance=2000,
    category="pois_health",
    num_pois=1,
)
n["pois"] = n["pois"] + (nearest_pois <= dist).sum(axis=1)

nearest_pois = network.nearest_pois(
    distance=2000,
    category="pois_basic",
    num_pois=1,
)
n["pois"] = n["pois"] + (nearest_pois <= dist).sum(axis=1)
#n["pois"] = n2 + n1


In [ ]:
fig, ax = ox.plot.plot_graph(
    G,
    node_size=0,
    edge_color="#afdffe",
    edge_linewidth=0.6,
    bgcolor="#1a1a1a",
    show=False,
    close=False,
    figsize=(16,14)
)

vmin = n["pois"].min()
vmax = n["pois"].max()

n.plot(
    ax=ax,
    column="pois",
    cmap="plasma",
    markersize=0.5,
    alpha=0.8,
    legend=True,
    legend_kwds={
        "shrink": 0.5,
        "label": f"Number of pois ≤ {dist} m",
        "orientation": "vertical"
    },
    vmin=0,
    vmax=vmax
)

plt.show()